# Anexo SAX

En el presente Jupyter notebook se pretender aportar una visión más amplia sobre el uso de SAX (_Simple API to Xml_) en Python ().

De una forma resumida lo que SAX es una arquitectura de procesamiento XML que nos proporciona la lectura de un archivo XML a medida que este se va leyendo, en cada elemento de un XML existen eventos (por ejemplo < cuenta > ..... </ cuenta>) sobre los que podemos desarrollar una serie de acciones. 

Como estamos leyendo el archivo XML por eventos, internamente, el sistema no genera ninguna estructura (como por ejemplo de tipo árbol) que nos permita guardar una jerarquía de los elementos del XML, esto es de gran interés cuando tengamos archivos XML muy grandes y que por lo tanto, para procesarlos, no sea interesante guardar toda una estructura interna del archivo en memoria, simplemente, ir recorriéndolo elemento a elemento.

Contra esta tipo de arquitectura se encuentra DOM, en la que se lee el archivo entero y se guarda internamente la jerarquía del mismo (por lo general en forma de árbol), de esta manera podemos iterar sobre sus diferentes elementos. Obviamente, el uso de poder almacenar todo un XML en memoria, hace que DOM sea mucho más rápido en procesamiento en archivos XML (aunque consuma mucha más memoria que SAX). 

Como ya se ha mencionado, vamos a trabajar a nivel de evento, por lo tanto, SAX nos brinda algunas de sus funciones para manejar eventos como es __<code>ContentHandler</code>__. Estos eventos, en un archivo XML comienzan desde el inicio de la lectura del archivo, pasando por los eventos que se generan en cada lectura de las etiquetas (elementos) < cuenta> del archivo y la información que contiene cada etiqueta, hasta el final de la lectura de la información de cada etiqueta, el final de cada etiqueta </ cuenta> y el cierre del propio documento. Como se muestra en la teoría el siguiente ejemplo de archivo xml desataría los siguientes eventos:


<?xml version=“1.0”?>

     <doc>

         <par>

         Hola Mundo

         </par>

     </doc>
     
     
* inicio de documento
* inicio de elemento doc
* inicio de elemento par
* caracteres Hola mundo
* fin de elemento par
* fin de elemento doc
* fin documento

Es importante recordar que cada evento se va almacenando en memoria, y por lo tanto, cuando un evento finaliza, este se libera en memoria y el manejador de eventos (ContentHandler) pasa al siguiente, asignándose la memoria al siguiente evento, es por ello que SAX es de sólo lectura.

Internamente el manejador de eventos se llama al principio y final de cada elemento. Si el analizador de eventos no se encuentra en modo espacio de nombres. (Ya hemos visto que un XML puede tener asociadas varias URIS con atributos comunes por lo que es recomendable utilizar espacios de nombres para denotar los atributos de cada URI). Es necesario crear los métodos __<code>startElement</code>__ incluyendo el nombre de la etiqueta a analizar y los atributos de la misma  y, __<code>endElement</code>__ incluyendo el nombre de la etiqueta.

Es muy importante conocer primeramente el método __<code>make_parser</code>__. Make Parser es el método encargado de crear el objeto analizador que va a ir recorriendo cada evento de nuestro XML y devolviéndolo al sistema (liberándo la memoria y asignándola a otro evento). 

Hemos visto que con make_parser creamos un analizador (primeramente vacío), para que este funcione, debemos asignarle un archivo xml con el que trabajar para ello existe la función __<code>parse()</code>__. Los principales argumentos que vamos a utilizar para esta función son:
* xmlfile: Nombre del archivo xml a analizar.
* contentHandler: El manejador de eventos que tenemos que crear con las funciones startElement y endElement (y más funciones si fueran necesarias).

Ahora, pasemos a la práctica. Lo primero que tenemos que tener en cuenta trabajando con SAX es la importación de la librería (por defecto, viene pre-instalada en distribuciones de Anaconda). Para ello, utilizaremos __<code>import xml.sax</code>__

In [1]:
import xml.sax

Como se ha mencionado anteriormente, lo primero que necesitamos para poder trabajar con SAX (además de importar la librería), es crear un analizador, para ello haremos uso de la función anteriormente mencionada <code>make_parser</code>

In [2]:
# Creamos un analizador de eventos
analizador_make_parser = xml.sax.make_parser()

Una vez que hemos creado un analizador de contenidos, podemos configurarlo, como hemos mencionado anteriormente, el analizador de eventos puede estar configurado para trabajar con espacios de nombres o directamente con las etiquetas del archivo xml. Para ello es importante mencionar en la documentación los diferentes elementos configurables de nuestro analizador. https://docs.python.org/3.6/library/xml.sax.handler.html

Tras realizar la revisión de la documentación específica del analizador, vemos que la función que nos permite activar o desactivar los espacios de nombres es <code>feature_namespaces</code>. Para poder añadir un tipo de configuración a nuestro analizador tenemos que hacerlo a través de <code>setProperty()</code>. Es importante reseñar que lo que estamos haciendo a través de setProperty es dada una configuración, asignarle un valor, para el caso de <code>feature_namespaces</code> es __TRUE__ o __FALSE__.

In [3]:
# Desactivamos los espacios de nombres ya que vamos a trabajar con etiquetas.
analizador_make_parser.setFeature(xml.sax.handler.feature_namespaces,
                                 False)

Posteriormente, una vez que hemos finalizado de configurar y crear nuestro analizador, tenemos que crear nuestro manejador de eventos o __<code>ContentHandler</code>__. Los principales objetos de un Content Handler son (Nota: cualesquiera de estos objetos tiene que ser definido por nosotros mismos, la lista completa se encuentra en: https://docs.python.org/3.6/library/xml.sax.handler.html):
* __setDocumentLocator__: Para localizar el origen de los eventos en el documento
* __startDocument__: Notifica cuando comienza a procesarse el documento.
* __endDocument__: Notifica cuando finaliza el procesamiento del documento.
* __startPrefixMapping__: Define el alcance de la asignación del espacio de nombres.
* __endPrefixMapping__: Finaliza la asignación de un espacio de nombres.
* __startElement__: Señala el inicio de un documento en modo sin espacio de nombres (lo vamos a utilizar).
* __endElement__: Señala el final de un elemento en modo sin espacio de nombres.
* __startElementNS__: Señala el comienzo de un elemento en modo espacio de nombres
* __endElementNS__: Señala el fin de un elemento en modo espacio de nombres
* __characters__: Encargado de obtener la información de los datos que se encuentren entre los atributos de un elemento.
* __ignorableWhitespace__: Ignorar espacios en blanco.
* __processingInstruction__: Este método se invoca solamente para notificar cada vez que se realice una instrucción de procesamiento 
* __skippedEntity__: Se activa si es necesario pasar por algo alguna eriqueta.


El procesamiento que vamos a realizar para nuestro ejemplo de la teoría es el siguiente:
* Notificar cuando comience el procesamiento.
* Iterar sobre cada etiqueta por evento
* Notificar cuando finalice el procesamiento.

Lo primero que tenemos que tener en cuenta es que todas las funciones que creemos tienen que estar dentro de una única clase que sea del tipo manejador de eventos o <code>ContentHandler</code> ya que será SAX el encargado de ir procesando el archivo XML en función del manejador de eventos que le pasemos como parámetro.

Para definir la clase, hemos de recordar que en Python, las clases tienen atributos propios de clase, estos atributos serán los datos que contiene el archivo XML, si observamos el XML catagologos.xml podemos ver que tenemos los siguientes atributos:
* Titulo
* Fecha
* Autor

Creamos la clase ManejadorEventosCatalogo, para definir los atributos propios de la clase tenemos que crear la función init, cada atributo que definimos viene precedido del prefijo self. o lo que es lo mismo, atributo propio.

In [4]:
# Creamos ContentHandler
class ManejadorEventosCatalogo(xml.sax.ContentHandler):
    # Definimos los atributos propios
    def __init__(self):
        self.titulo = ""
        self.fecha = ""
        self.autor = ""

Como para cada elmento (etiqueta) Libro que aparezca en el XML vamos a recibir un contenido diferente tanto para título, fecha y autor, el valor por defecto de estos atributos será vacío. Hasta que sobreescribamos el contenido de estas variables.

El único elemento iterable que aparece en nuestro XML es Libro, por lo que para poder trabajar con el elemento Libro tenemos que definir la función start Element que va a recibir los atributos propios (cuando se vayan a utilizar), el nombre de la etiqueta y el identificador de la etiqueta, que en este caso es el isbn.

Como cada vez que iteramos vamos a estar procesando un libro diferente, vamos a crear un nuevo atributo propio denominado libro_iterable.

In [5]:
# Creamos ContentHandler
class ManejadorEventosCatalogo(xml.sax.ContentHandler):
    # Definimos los atributos propios
    def __init__(self):
        self.titulo = ""
        self.fecha = ""
        self.autor = ""
        self.libro_iterable = ""
    
    def startElement(self,etiqueta,isbn):
        self.libro_iterable = etiqueta
        if etiqueta == "Libro":
            print("*** LIBRO ***")
            isbn=isbn["isbn"]
            print("isbn:",isbn)

Cuando estamos iterando en cada libro, lo que en realidad estamos haciendo es procesar un evento que va a contener toda la información del libro, por lo que, antes de que el analizador libere la memoria del evento para pasar al siguiente, tenemos que guardar la información del libro para poder mostrarla por pantalla. Para ello, lo que vamos a hacer es que cuando tenermine el procesamiento de un elemento guarde su información, para ello hemos visto que al igual que <code>startElement</code>, existe <code>endElement</code>. 

De nuevo, vemos que el título, fecha y autor son diferentes para cada libro, es decir, vamos a iterar sobre cada sub-elemento de libro, para ello, vamos a hacer uso de la variable libro_iterable que está recibiendo el valor de la etiqueta sobre la que estamos iterando.

In [6]:
# Creamos ContentHandler
class ManejadorEventosCatalogo(xml.sax.ContentHandler):
    # Definimos los atributos propios
    def __init__(self):
        self.titulo = ""
        self.fecha = ""
        self.autor = ""
        self.libro_iterable = ""
    
    def startElement(self,etiqueta,isbn):
        self.libro_iterable = etiqueta
        if etiqueta == "Libro":
            print("*** LIBRO ***")
            isbn=isbn["isbn"]
            print("isbn:",isbn)
        else:
            pass
            
    def endElement(self,etiqueta):
        if self.libro_iterable == "titulo":
            print("Titulo:",self.titulo)
        elif self.libro_iterable == "fecha":
            print("Fecha:", self.fecha)
        elif self.libro_iterable == "autor":
            print("Autor:", self.autor)
        else:
            pass
        # Reiniciamos la variable para que la siguiente vuelva a tomar el nombre de la etiqueta
        self.libro_iterable=""

Finalmente, para mostrar por pantalla el contenido de las sub-etiquetas titulo, fecha y autor, vamos a ir cambiando el valor de la variable libro_iterable por el nombre de estas sub-etiquetas, para que cuando ContenHandler llame a la función <code>endElement</code>, muestre por pantalla su contenido.

Adicionalmente vamos a añadir un mensaje de inicio/fin de procesamiento con las funciones <code>startDocument</code> y <code>endDocument</code>

In [7]:
# Creamos ContentHandler
class ManejadorEventosCatalogo(xml.sax.ContentHandler):
    # Definimos los atributos propios
    def __init__(self):
        self.titulo = ""
        self.fecha = ""
        self.autor = ""
        self.libro_iterable = ""
    
    def startElement(self,etiqueta,isbn):
        self.libro_iterable = etiqueta
        if etiqueta == "Libro":
            print("*** LIBRO ***")
            isbn=isbn["isbn"]
            print("isbn:",isbn)
        else:
            pass
            
    def endElement(self,etiqueta):
        if self.libro_iterable == "titulo":
            print("Titulo:",self.titulo)
        elif self.libro_iterable == "fecha":
            print("Fecha:", self.fecha)
        elif self.libro_iterable == "autor":
            print("Autor:", self.autor)
        else:
            pass
        # Reiniciamos la variable para que la siguiente vuelva a tomar el nombre de la etiqueta
        self.libro_iterable=""
        
    def characters(self, contenido):
        # Si el nombre es titulo la etiqueta cambiará a titulo y en endElement
        #   imprimiremos su valor
        if self.libro_iterable == "titulo":
            self.titulo = contenido
        elif self.libro_iterable == "fecha":
            self.fecha = contenido
        elif self.libro_iterable == "autor":
            self.autor = contenido
        else:
            pass
            
    def startDocument(self):
        print('----------------------------------------')
        print('Comienzo en el procesamiento del archivo')
        print('----------------------------------------\n')
        
    def endDocument(self):
        print('----------------------------------------')
        print('Fin del procesamiento del archivo xml')
        print('----------------------------------------')

Ahora que ya hemos definido nuestor manejador de eventos, lo iniciamos y lo pasamos a una variable.

In [8]:
Handler = ManejadorEventosCatalogo()

Vemos que pese a que hemos iniciado nuestro manejador de eventos, como aún no tiene ningún analizador asignnado, no devuelve nada. Esto podemos confirmarlo con <code>getContentHandler</code>

In [9]:
analizador_make_parser.getContentHandler()

Para asignar el manejador de eventos a un analizador utilizamos <code>setContentHander</code>

In [10]:
analizador_make_parser.setContentHandler(Handler)

In [11]:
analizador_make_parser.getContentHandler()

Finalmente, pasa que el analizador, analice un archivo tenemos que utilizar la función <code>parse</code> para indicarle el archivo XML que vamos a procesar.

In [12]:
analizador_make_parser.parse("datos.xml")

----------------------------------------
Comienzo en el procesamiento del archivo
----------------------------------------

*** LIBRO ***
isbn: 0-596-00128-2
Titulo: Python y XML
Fecha: Diciembre 2001
Autor: Pepito Perez
*** LIBRO ***
isbn: 0-596-15810-6
Titulo: Programacion avanzada de XML
Fecha: Octoubre 2010
Autor: Juan Garcia
*** LIBRO ***
isbn: 0-596-15806-8
Titulo: Aprendiendo Java
Fecha: Septiembre 2009
Autor: Juan Garcia
*** LIBRO ***
isbn: 0-596-15808-4
Titulo: Python para moviles
Fecha: Octubre 2009
Autor: Pepito Perez
*** LIBRO ***
isbn: 0-596-00797-3
Titulo: R para estadistica
Fecha: Marzo 2005
Autor: Juan
Autor: Pepe
Autor: Isabel
*** LIBRO ***
isbn: 0-596-10046-9
Titulo: Python en 100 paginas
Fecha: Julio 2006
Autor: Julia
----------------------------------------
Fin del procesamiento del archivo xml
----------------------------------------


# Ejemplo completo

In [13]:
# Creamos ContentHandler
class ManejadorEventosCatalogo(xml.sax.ContentHandler):
    # Definimos los atributos propios
    def __init__(self):
        self.titulo = ""
        self.fecha = ""
        self.autor = ""
        self.libro_iterable = ""
    
    def startElement(self,etiqueta,isbn):
        self.libro_iterable = etiqueta
        if etiqueta == "Libro":
            print("*** LIBRO ***")
            isbn=isbn["isbn"]
            print("isbn:",isbn)
        else:
            pass
            
    def endElement(self,etiqueta):
        if self.libro_iterable == "titulo":
            print("Titulo:",self.titulo)
        elif self.libro_iterable == "fecha":
            print("Fecha:", self.fecha)
        elif self.libro_iterable == "autor":
            print("Autor:", self.autor)
        else:
            pass
        # Reiniciamos la variable para que la siguiente vuelva a tomar el nombre de la etiqueta
        self.libro_iterable=""
        
    def characters(self, contenido):
        # Si el nombre es titulo la etiqueta cambiará a titulo y en endElement
        #   imprimiremos su valor
        if self.libro_iterable == "titulo":
            self.titulo = contenido
        elif self.libro_iterable == "fecha":
            self.fecha = contenido
        elif self.libro_iterable == "autor":
            self.autor = contenido
        else:
            pass
            
    def startDocument(self):
        print('----------------------------------------')
        print('Comienzo en el procesamiento del archivo')
        print('----------------------------------------\n')
        
    def endDocument(self):
        print('----------------------------------------')
        print('Fin del procesamiento del archivo xml')
        print('----------------------------------------')

In [14]:
if ( __name__ == "__main__"):
    # Creamos un analizador de eventos
    analizador_make_parser = xml.sax.make_parser()
    # Desactivamos los espacios de nombres ya que vamos a trabajar con etiquetas.
    analizador_make_parser.setFeature(xml.sax.handler.feature_namespaces,
                                     False)

    Handler = ManejadorEventosCatalogo()
    analizador_make_parser.setContentHandler(Handler)
   
    analizador_make_parser.parse("datos.xml")

----------------------------------------
Comienzo en el procesamiento del archivo
----------------------------------------

*** LIBRO ***
isbn: 0-596-00128-2
Titulo: Python y XML
Fecha: Diciembre 2001
Autor: Pepito Perez
*** LIBRO ***
isbn: 0-596-15810-6
Titulo: Programacion avanzada de XML
Fecha: Octoubre 2010
Autor: Juan Garcia
*** LIBRO ***
isbn: 0-596-15806-8
Titulo: Aprendiendo Java
Fecha: Septiembre 2009
Autor: Juan Garcia
*** LIBRO ***
isbn: 0-596-15808-4
Titulo: Python para moviles
Fecha: Octubre 2009
Autor: Pepito Perez
*** LIBRO ***
isbn: 0-596-00797-3
Titulo: R para estadistica
Fecha: Marzo 2005
Autor: Juan
Autor: Pepe
Autor: Isabel
*** LIBRO ***
isbn: 0-596-10046-9
Titulo: Python en 100 paginas
Fecha: Julio 2006
Autor: Julia
----------------------------------------
Fin del procesamiento del archivo xml
----------------------------------------
